In [ ]:
import numpy as np
import pandas as pd
import csv
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, RepeatVector, Dense
from tensorflow.keras.layers import Reshape
from sklearn.preprocessing import MinMaxScaler

##Peaks Normalization and Log Transformation

In [ ]:
data = []
with open('CrestFactor.csv') as f:
    reader = csv.reader(f)
    next(reader) # skip header
    for row in reader:
        float_row = []
        for x in row[1:]:
            if x == '':
                float_row.append(np.nan)
            else:
                float_row.append(float(x))
        data.append(float_row)

# Replace NaN with 0
data = [np.nan_to_num(x) for x in data]

data=pd.DataFrame(data)

data = data.iloc[76:91]
timesteps = 15
n_features = data.shape[1]

In [ ]:
data

,0,1,2,3,4,5,6,7,8,9,10
76,4.73768,4.550594,3.085939,3.761257,3.424900,2.324135,4.226144,3.231132,3.885500,2.859678,2.788577
77,4.60983,4.410248,3.924447,2.815428,2.656854,2.021950,3.368819,2.502890,2.486855,3.934300,3.137370
78,2.27748,4.224191,4.736614,2.675823,2.348434,2.306741,3.022760,2.736767,3.427256,3.840598,3.256828
79,2.75521,4.340199,8.448929,2.683527,2.731767,2.290176,2.839650,3.081490,3.074391,3.317951,2.944420
80,2.65993,5.259423,6.737805,2.427472,2.739425,2.440444,3.962506,2.770874,2.654393,2.510645,2.768448
81,3.20969,3.720177,3.881008,2.469332,2.763350,2.157739,3.686598,2.565808,2.571747,6.232784,2.935981
82,2.66146,3.261082,3.753092,2.746995,2.955256,2.088982,2.928486,2.843193,2.790592,2.631454,3.887653
83,3.54430,4.912873,2.834177,2.735943,3.112673,2.342870,4.718598,3.060827,3.266568,3.007311,2.816620
84,2.94843,7.137865,3.999931,2.766169,2.868385,3.198738,4.924803,2.985579,2.563372,2.688332,2.934326
85,2.47071,2.961327,6.424623,2.599626,2.936126,2.566126,3.068718,2.975834,3.278681,2.618233,3.726993


In [ ]:
inputs = Input(shape=(timesteps, 1))
x = LSTM(15)(inputs)
x = Dense(10)(x)
# x = Dense(5)(x)
encoded = Dense(1)(x)

# decoded = Dense(16)(encoded)
# decoded = Dense(32)(decoded)
# decoded = Dense(61)(decoded)
# decoded = Reshape((61, 1))(decoded)
# decoded = LSTM(1, return_sequences=True)(decoded)
decoded = Dense(timesteps)(encoded)
decoded = Reshape((timesteps, 1))(decoded)
decoded = LSTM(15, return_sequences=True)(decoded)
decoded = Dense(10)(decoded)
# decoded = Dense(5)(decoded)
decoded = Dense(1)(decoded)

latent_space_model = Model(inputs, encoded)
autoencoder = Model(inputs, decoded)
autoencoder.compile(optimizer='adam', loss='mae')


In [ ]:
autoencoder.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 15, 1)]           0         
                                                                 
 lstm_4 (LSTM)               (None, 15)                1020      
                                                                 
 dense_10 (Dense)            (None, 10)                160       
                                                                 
 dense_11 (Dense)            (None, 5)                 55        
                                                                 
 dense_12 (Dense)            (None, 1)                 6         
                                                                 
 dense_13 (Dense)            (None, 15)                30        
                                                                 
 reshape_2 (Reshape)         (None, 15, 1)             0   

In [ ]:
for i in range(n_features):
    print(f'Training on column: {i+1}/{n_features}')
    x_train = data.iloc[:, i].values.reshape(-1, timesteps, 1)

    autoencoder.fit(x_train, x_train, epochs=20, batch_size=20,verbose=0)

# Save model weights
autoencoder.save_weights('autoencoder_weights.h5')

Training on column: 1/11
Training on column: 2/11
Training on column: 3/11
Training on column: 4/11
Training on column: 5/11
Training on column: 6/11
Training on column: 7/11
Training on column: 8/11
Training on column: 9/11
Training on column: 10/11
Training on column: 11/11


In [ ]:
def predict_sequence_and_latent_space(input_sequence):
    input_sequence = np.array(input_sequence)
    input_sequence = input_sequence.reshape(1, -1, 1)
    print(input_sequence.shape)
    predicted_sequence = autoencoder.predict(input_sequence)
    latent_space_output = latent_space_model.predict(input_sequence)
    return predicted_sequence.reshape(-1), latent_space_output[0][0]

column_to_predict = 0
input_sequence = data.iloc[:, column_to_predict].values.tolist()
predictions, latent_space_output = predict_sequence_and_latent_space(input_sequence)

print("Original Sequence:")
print(input_sequence)

print("Predicted Sequence:")
print(predictions)

print("Latent Space Output:")
print(latent_space_output)

(1, 15, 1)
1/1 [==============================] - 0s 266ms/step
Original Sequence:
[4.73768, 4.60983, 2.27748, 2.75521, 2.65993, 3.20969, 2.66146, 3.5443, 2.94843, 2.47071, 2.20765, 3.19376, 2.64116, 3.09031, 2.82989]
Predicted Sequence:
[1.7655408 1.7330223 2.586159  2.5771353 3.0253808 3.110044  3.2030427
 3.2026563 3.2379072 3.3525615 3.2362142 2.9982896 3.0105746 2.9352984
 2.642624 ]
Latent Space Output:
8.14011


In [ ]:
import pandas as pd

# Convert the predicted sequence back to a DataFrame
predictions_df = pd.DataFrame(predictions, columns=['Predicted Sequence'])

# Get the original DataFrame
original_df = pd.DataFrame(input_sequence, columns = ['Original Sequence'])

# Get the column to predict (column_to_predict) and the corresponding column name
# column_name = original_df.columns[column_to_predict]

# Append the predicted sequence to the original DataFrame
original_df['Predicted Sequence'] = predictions_df

# Save the DataFrame to a CSV file
original_df.to_csv('predicted_sequences.csv', index=False)

print("Original Sequence and Predicted Sequence saved to 'predicted_sequences.csv'")


Original Sequence and Predicted Sequence saved to 'predicted_sequences.csv'


In [ ]:
num_columns = 11  # Total number of columns in your data

def predict_and_get_latent_output(input_sequence, column_idx):
    input_sequence = np.array(input_sequence)
    input_sequence = input_sequence.reshape(1, -1, 1)
    predicted_sequence = autoencoder.predict(input_sequence)
    latent_space_output = latent_space_model.predict(input_sequence)
    return predicted_sequence.reshape(-1), latent_space_output.reshape(-1)

# Create an array to store the latent_space_op values for each column
latent_space_op_array = []

# Loop over each column
for column_idx in range(num_columns):
    input_sequence = data.iloc[:, column_idx].values.tolist()
    predictions, latent_space_output = predict_and_get_latent_output(input_sequence, column_idx)
    latent_space_op_array.append(latent_space_output)

print(latent_space_op_array)

1/1 [==============================] - 0s 16ms/step
[array([8.14011], dtype=float32), array([8.266882], dtype=float32), array([8.272763], dtype=float32), array([8.059239], dtype=float32), array([8.026466], dtype=float32), array([8.013733], dtype=float32), array([8.47614], dtype=float32), array([8.067782], dtype=float32), array([8.119508], dtype=float32), array([8.069066], dtype=float32), array([8.176468], dtype=float32)]


In [ ]:
latent_space_op_array

[array([8.14011], dtype=float32),
 array([8.266882], dtype=float32),
 array([8.272763], dtype=float32),
 array([8.059239], dtype=float32),
 array([8.026466], dtype=float32),
 array([8.013733], dtype=float32),
 array([8.47614], dtype=float32),
 array([8.067782], dtype=float32),
 array([8.119508], dtype=float32),
 array([8.069066], dtype=float32),
 array([8.176468], dtype=float32)]

Data saved to output_data.csv


In [ ]:
data = [['Latent Space Output for each column:']]
for item in latent_space_op_array:
    data.append([item[0]])

# Write data to CSV file
csv_filename = 'latent_space_output.csv'
with open(csv_filename, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(data)
